## Step by step Creation of the different functions

### 1

In [1]:
import termcolor
import mysql.connector
import time
from prettytable import PrettyTable


import csv
import os

In [2]:
def connect_to_mysql(config, attempts=3, delay=2):
    attempt = 1
    # Implement a reconnection routine
    while attempt < attempts + 1:
        try:
            no_dict=""
            if type(config)!= dict:
                no_dict="Not dict type"
            connector=mysql.connector.connect(**config)
            termcolor.cprint("Connected successfully....","green", attrs=['bold'], end='\n')
            return connector
        except (mysql.connector.Error, IOError, Exception) as e:
            if (attempts is attempt):
                # Attempts to reconnect failed; returning None
                termcolor.cprint("Failed to connect, exiting without a connection:","red", attrs=['bold'], end=' ')
                print(e, no_dict)
                return None
            # progressive reconnect delay
            time.sleep(delay ** attempt)
            attempt += 1
    return None


In [3]:
config={
    'host':'localhost',
    'user':'root',
    'password':'Ravi@46931',
    'database':'firstsql'
}

## Connect to MySQL server
conn=connect_to_mysql(config, attempts=3)

Connected successfully....


In [4]:
conn

In [5]:
def execute_query(connection, query):
    # Verify that connection is MySQL connection object
    if not isinstance(connection, mysql.connector.connection_cext.CMySQLConnection):
        termcolor.cprint("Error Connection Object:","red", attrs=['bold'], end=' ')
        print("Connection is not a MySQL connection object.")
        return
    try:
        cursor=connection.cursor()
        cursor.execute(query)
        try:
            rows = cursor.fetchall()
            column_names = [desc[0] for desc in cursor.description]
            # Display results using PrettyTable
            if rows:
                table = PrettyTable(column_names)
                table.align = 'l'
                for row in rows:
                    table.add_row(row)
                termcolor.cprint("Query executed successfully....","green", attrs=['bold'], end='\n')
                print(table)
            else:
                termcolor.cprint("No results found.","red", attrs=['bold'], end=' ')
                
        except Exception as e:
            termcolor.cprint("Query:","green", attrs=['bold'], end=' ')
            termcolor.cprint(f"\"{query}\"","blue", attrs=['bold'], end=' ')
            termcolor.cprint("executed successfully.","green", attrs=['bold'], end=' ')
            return
    except Exception as e:
        termcolor.cprint("Error executing query:","red", attrs=['bold'], end=' ')
        print(e)

In [65]:
query="use book_shop"

query="show databases"
query="select database()"
query="show tables"
query="use lit"
query="select * from cats"
execute_query(conn, query)              

Query executed successfully....
+--------+------+
| name   | age  |
+--------+------+
| JELLY  | 5    |
| JEGH   | 3    |
| KOGH   | 3    |
| reu    | 5    |
| xavior | 7    |
| isil   | 8    |
| lilly  | None |
| None   | None |
| JELLY  | 5    |
| None   | 15   |
| None   | None |
| Raki   | 15   |
| Jolly  | 35   |
| Polly  | 25   |
| Poll   | 23   |
| Toll   | 2    |
| Rakou  | 15   |
| Rakou  | 15   |
| Toying | 23   |
| Kohl   | 2    |
| Val    | 12   |
| Kohl   | 2    |
| Val    | 12   |
| Tom    | 2    |
| Sam    | 12   |
| Tom    | 2    |
| Sammy  | 11   |
| Helan  | 2    |
| nemao  | 11   |
+--------+------+


In [7]:
cursor=conn.cursor()
query=f"select * from cats"
cursor.execute(query)
cursor.fetchall()

[('JELLY', 5),
 ('JEGH', 3),
 ('KOGH', 3),
 ('reu', 5),
 ('xavior', 7),
 ('isil', 8),
 ('lilly', None),
 (None, None),
 ('JELLY', 5),
 (None, 15),
 (None, None),
 ('Raki', 15),
 ('Jolly', 35),
 ('Polly', 25),
 ('Poll', 23),
 ('Toll', 2),
 ('Rakou', 15),
 ('Rakou', 15),
 ('Toying', 23),
 ('Kohl', 2),
 ('Val', 12),
 ('Kohl', 2),
 ('Val', 12),
 ('Tom', 2),
 ('Sam', 12),
 ('Tom', 2),
 ('Sammy', 11)]

In [7]:
def insert_data(connection, table_name, values, db_name =""):
    # check if the connection is mysql connector
    if not isinstance(connection, mysql.connector.connection_cext.CMySQLConnection):
        termcolor.cprint("Error Connection Object:","red", attrs=['bold'], end=' ')
        print("Connection is not a MySQL connection object.")
        return
    
    # if database is not given hence find the current database
    if db_name=="":
        cursor=connection.cursor()
        query_select="select database()"
        cursor.execute(query_select)
        db=cursor.fetchall()
        db_name=db[0][0]
        print(f"Your database: '{db_name}'")
    # if user has given database 
    else:
        try:
            cursor=connection.cursor()
            query_db=f"use {db_name}"
            cursor.execute(query_db)
            print(f"Your database: {db_name}")
        except Exception as e:
            termcolor.cprint("Error using db_name:","red", attrs=['bold'], end=' ')
            print(e)
            return
    
    # Fetching the tables of the database  
    try: 
        query_tables='show tables'
        cursor.execute(query_tables)
        tables=cursor.fetchall()
        list_table=[table[0]  for table in tables]

        if table_name in list_table:
            termcolor.cprint("Your table:","green", attrs=['bold'], end=' ')
            termcolor.cprint(f"'{table_name}'", "blue", attrs=['bold'], end=' ')
            termcolor.cprint(", present in the database", end=' ')
            termcolor.cprint(f"'{db_name}'", "blue", attrs=['bold'], end='\n')

        else:
            termcolor.cprint("Table not found:", "red", attrs=['bold'], end=' ')
            print(f"'{table_name}' is not present in the database '{db_name}'.")
            print(f"Select one of the following tables ", end='')
            termcolor.cprint(f"{list_table}", attrs=['bold'], end=' ')
            print("or create new table or use another database.")
            return
    except Exception as e:
        termcolor.cprint("Error fetching the tables:","red", attrs=['bold'], end=' ')
        print(e)
        return
        
    
    
    # if table is present in the database
    try:   
        query_table=f"select * from {table_name}"
        cursor.execute(query_table)
        cursor.fetchall()
        desc=cursor.description
        columns=[t[0] for t in desc]

        flag=input("Enter yes if the first value of argument is 'id' and you have not provided: ").lower()
        if flag=='y' or flag=='yes':
            columns=columns[1:]
        d=str(tuple(columns))
        column_table=d.replace('\'','')
        
    except Exception as e:
        termcolor.cprint("Error fetching column names:","red", attrs=['bold'], end=' ')
        print(e)
        return
        
    # Writing query
    query= f"INSERT INTO {table_name} {column_table} VALUES ({', '.join('%s' for _ in columns)})"
    try:
        tmp=0
        while(tmp<3):
            flag_insert=input("Are you inserting more than one entry?(y/n) ").lower()
            if flag_insert=='y':
                cursor.executemany(query, values)
                break
            elif flag_insert=='n':    
                cursor.execute(query, values)
                break
            else:
                print("Incorrect option..")
            tmp+=1
        if tmp==3:
            termcolor.cprint("Program exited without inserting","magenta", attrs=['bold'], end=' ')
            return
        connection.commit()
        termcolor.cprint("Inserted successfully....", 'green', attrs=['bold'], end=' ')
    except Exception as e:
        termcolor.cprint("Error inserting data:","red", attrs=['bold'], end=' ')
        print(e)

In [8]:
# db_name='hyj'
table_name='cats'
values=[
    ('Tom', 2),
    ("Sammy",11)
]
insert_data(conn, table_name, values)

Your database: 'lit'
Your table: 'cats' , present in the database 'lit'
Enter yes if the first value of argument is 'id' and you have not provided: q
Are you inserting more than one entry?(y/n) qq
Incorrect option..
Are you inserting more than one entry?(y/n) 
Incorrect option..
Are you inserting more than one entry?(y/n) q
Incorrect option..
Program exited without inserting 

In [138]:
conn=connect_to_mysql(config, attempts=3)

Connected successfully.... 

In [58]:
query="select database()"
query="use book_shop"
query='show databases'
# query='desc cleint'
query='show tables'
query='select * from cats'
execute_query(conn, query)

Query executed successfully....
+--------+------+
| name   | age  |
+--------+------+
| JELLY  | 5    |
| JEGH   | 3    |
| KOGH   | 3    |
| reu    | 5    |
| xavior | 7    |
| isil   | 8    |
| lilly  | None |
| None   | None |
| JELLY  | 5    |
| None   | 15   |
| None   | None |
| Raki   | 15   |
| Jolly  | 35   |
| Polly  | 25   |
| Poll   | 23   |
| Toll   | 2    |
| Rakou  | 15   |
| Rakou  | 15   |
| Toying | 23   |
| Kohl   | 2    |
| Val    | 12   |
| Kohl   | 2    |
| Val    | 12   |
| Tom    | 2    |
| Sam    | 12   |
| Tom    | 2    |
| Sammy  | 11   |
| Helan  | 2    |
| nemao  | 11   |
+--------+------+


In [9]:
def save_data(connection, table_name, db_name=""):
    # check if the connection is mysql connector
    if not isinstance(connection, mysql.connector.connection_cext.CMySQLConnection):
        termcolor.cprint("Error Connection Object:","red", attrs=['bold'], end=' ')
        print("Connection is not a MySQL connection object.")
        return
    
    if db_name=="":
        cursor=connection.cursor()
        query_select="select database()"
        cursor.execute(query_select)
        db=cursor.fetchall()
        db_name=db[0][0]
    else:
        try:
            cursor=connection.cursor()
            query_db=f"use {db_name}"
            cursor.execute(query_db)
        except Exception as e:
            termcolor.cprint("Error using db_name:","red", attrs=['bold'], end=' ')
            print(e)
            return
    
    try: 
        query_tables='show tables'
        cursor.execute(query_tables)
        tables=cursor.fetchall()
        list_table=[table[0]  for table in tables]

        if table_name not in list_table:
            termcolor.cprint("Table not found:", "red", attrs=['bold'], end=' ')
            print(f"'{table_name}' is not present in the database '{db_name}'.")
            print(f"Select one of the following tables ", end='')
            termcolor.cprint(f"{list_table}", attrs=['bold'], end=' ')
            print("or create new table or use another database.")
            return
            
    except Exception as e:
        termcolor.cprint("Error fetching the tables:","red", attrs=['bold'], end=' ')
        print(e)
        return
    
    try:
        query = f"SELECT * FROM {table_name}"
        cursor.execute(query)
        rows = cursor.fetchall()
        column_names = [desc[0] for desc in cursor.description]
        csv_file_name =input("Enter the filename: ")
        # Create the full path for the output CSV file
        file_path=os.path.join(os.getcwd(), csv_file_name)
        with open(file_path, 'w', newline='') as csv_file:
            csv_writer = csv.writer(csv_file)
            csv_writer.writerow(column_names)
            csv_writer.writerows(rows)
        if os.path.exists(csv_file_name):
            termcolor.cprint("File:", 'green', attrs=['bold'], end=' ')
            termcolor.cprint(f"'{csv_file_name}'", 'blue', attrs=['bold'], end=' ')
            termcolor.cprint("saved successfully....", 'green', attrs=['bold'])
            
    except Exception as e:
        termcolor.cprint("Error saving data:","red", attrs=['bold'], end=' ')
        print(e)

In [11]:
table_name='cats'
db_name='lit'
save_data(conn, table_name, db_name)

Enter the filename: oi.csv
File: 'oi.csv' saved successfully....


In [ ]:
## Give name of the database or default 
## Give the correct name of the table

In [51]:
# Helper functions
def fetch_db(connection,db_name):
    if db_name=="":
        cursor=connection.cursor()
        query_select="select database()"
        cursor.execute(query_select)
        db=cursor.fetchall()
        db_name=db[0][0]
        return cursor,db_name
    else:
        try:
            cursor=connection.cursor()
            query_db=f"use {db_name}"
            cursor.execute(query_db)
            return cursor,db_name
        except Exception as e:
            termcolor.cprint("Error using db_name:","red", attrs=['bold'], end=' ')
            print(e,end='')
            return _,None

def fetch_tables(cursor):
    query_tables='show tables'
    cursor.execute(query_tables)
    tables=cursor.fetchall()
    list_table=[table[0]  for table in tables]
    return list_table

In [52]:
def save_dta(connection, table_name, db_name=""):
    # check if the connection is mysql connector
    if not isinstance(connection, mysql.connector.connection_cext.CMySQLConnection):
        termcolor.cprint("Error Connection Object:","red", attrs=['bold'], end=' ')
        print("Connection is not a MySQL connection object.")
        return
    
    cursor,db_name=fetch_db(connection,db_name)
    if db_name==None:
        return
    
    try: 
        list_table=fetch_tables(cursor)
        
        if table_name not in list_table:
            termcolor.cprint("Table not found:", "red", attrs=['bold'], end=' ')
            print(f"'{table_name}' is not present in the database '{db_name}'.")
            print(f"Select one of the following tables ", end='')
            termcolor.cprint(f"{list_table}", attrs=['bold'], end=' ')
            print("or create new table or use another database.")
            return
            
    except Exception as e:
        termcolor.cprint("Error fetching the tables:","red", attrs=['bold'], end=' ')
        print(e)
        return
    
    try:
        query = f"SELECT * FROM {table_name}"
        cursor.execute(query)
        rows = cursor.fetchall()
        column_names = [desc[0] for desc in cursor.description]
        csv_file_name =input("Enter the filename: ")
        # Create the full path for the output CSV file
        file_path=os.path.join(os.getcwd(), csv_file_name)
        with open(file_path, 'w', newline='') as csv_file:
            csv_writer = csv.writer(csv_file)
            csv_writer.writerow(column_names)
            csv_writer.writerows(rows)
        if os.path.exists(csv_file_name):
            termcolor.cprint("File:", 'green', attrs=['bold'], end=' ')
            termcolor.cprint(f"'{csv_file_name}'", 'blue', attrs=['bold'], end=' ')
            termcolor.cprint("saved successfully....", 'green', attrs=['bold'])
            
    except Exception as e:
        termcolor.cprint("Error saving data:","red", attrs=['bold'], end=' ')
        print(e)

In [53]:
table_name='cats'
db_name='lit'
save_dta(conn, table_name,db_name)

Enter the filename: t
File: 't' saved successfully....


In [56]:
def insert_dta(connection, table_name, values, db_name =""):
    # check if the connection is mysql connector
    if not isinstance(connection, mysql.connector.connection_cext.CMySQLConnection):
        termcolor.cprint("Error Connection Object:","red", attrs=['bold'], end=' ')
        print("Connection is not a MySQL connection object.")
        return
    
    # if database is not given hence find the current database
    cursor,db_name=fetch_db(connection,db_name)
    if db_name==None:
        return
    else:
        print(f"Your database: '{db_name}'")
    
    # Fetching the tables of the database  
    try: 
        list_table=fetch_tables(cursor)

        if table_name in list_table:
            termcolor.cprint("Your table:","green", attrs=['bold'], end=' ')
            termcolor.cprint(f"'{table_name}'", "blue", attrs=['bold'], end=' ')
            termcolor.cprint(", present in the database", end=' ')
            termcolor.cprint(f"'{db_name}'", "blue", attrs=['bold'], end='\n')

        else:
            termcolor.cprint("Table not found:", "red", attrs=['bold'], end=' ')
            print(f"'{table_name}' is not present in the database '{db_name}'.")
            print(f"Select one of the following tables ", end='')
            termcolor.cprint(f"{list_table}", attrs=['bold'], end=' ')
            print("or create new table or use another database.")
            return
    except Exception as e:
        termcolor.cprint("Error fetching the tables:","red", attrs=['bold'], end=' ')
        print(e)
        return
        
    
    
    # if table is present in the database
    try:   
        query_table=f"select * from {table_name}"
        cursor.execute(query_table)
        cursor.fetchall()
        desc=cursor.description
        columns=[t[0] for t in desc]

        flag=input("Enter yes if the first value of argument is 'id' and you have not provided: ").lower()
        if flag=='y' or flag=='yes':
            columns=columns[1:]
        d=str(tuple(columns))
        column_table=d.replace('\'','')
        
    except Exception as e:
        termcolor.cprint("Error fetching column names:","red", attrs=['bold'], end=' ')
        print(e)
        return
        
    # Writing query
    query= f"INSERT INTO {table_name} {column_table} VALUES ({', '.join('%s' for _ in columns)})"
    try:
        tmp=0
        while(tmp<3):
            flag_insert=input("Are you inserting more than one entry?(y/n) ").lower()
            if flag_insert=='y':
                cursor.executemany(query, values)
                break
            elif flag_insert=='n':    
                cursor.execute(query, values)
                break
            else:
                print("Incorrect option..")
            tmp+=1
        if tmp==3:
            termcolor.cprint("Program exited without inserting","magenta", attrs=['bold'], end=' ')
            return
        connection.commit()
        termcolor.cprint("Inserted successfully....", 'green', attrs=['bold'], end=' ')
    except Exception as e:
        termcolor.cprint("Error inserting data:","red", attrs=['bold'], end=' ')
        print(e)

In [63]:
db_name='hyj'
table_name='cats'
values=[
    ('Helan', 2),
    ("nemao",11)
]
insert_dta(conn, table_name, values)

Your database: 'lit'
Your table: 'cats' , present in the database 'lit'
Enter yes if the first value of argument is 'id' and you have not provided: 
Are you inserting more than one entry?(y/n) y
Error inserting data: Parameters for query must be list or tuple.


In [33]:
# Coonection is verified
cursor = conn.cursor()


In [62]:
# 
table_name='cats'
quey = f"SELECT * FROM {table_name}"
quey

'SELECT * FROM cats'

In [63]:
cursor.execute(quey)

In [64]:
rows = cursor.fetchall()
rows

[('JELLY', 5),
 ('JEGH', 3),
 ('KOGH', 3),
 ('reu', 5),
 ('xavior', 7),
 ('isil', 8),
 ('lilly', None),
 (None, None),
 ('JELLY', 5),
 (None, 15),
 (None, None),
 ('Raki', 15),
 ('Jolly', 35),
 ('Polly', 25),
 ('Poll', 23),
 ('Toll', 2),
 ('Rakou', 15),
 ('Rakou', 15),
 ('Toying', 23),
 ('Kohl', 2),
 ('Val', 12),
 ('Kohl', 2),
 ('Val', 12),
 ('Tom', 2),
 ('Sam', 12)]

In [60]:
column_names = [desc[0] for desc in cursor.description]
column_names

['name', 'age']

In [51]:
csv_file_name = 'output.csv'
csv_file_name =input("Enter the filename: ")

Enter the filename: out.csv


In [61]:
if not os.path.exists(csv_file_name):
    print("Hi")
    # Create the full path for the output CSV file
    file_path=os.path.join(os.getcwd(), csv_file_name)
with open(file_path, 'w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(column_names)
    csv_writer.writerows(rows)
print(f"File '{file_path}' saved successfully.")

File 'E:\MLOps\ineuron_python_package(copy)\experiments\out.csv' saved successfully.


In [48]:
folder_path = 'outputf'
csv_file_name = 'output.csv'
file_path='outputf/output.csv'

In [65]:
t='Hi'
tables=['Hi', 'Ki', 'Ji']

In [67]:
if t not in tables:
    print("Yes")

In [63]:
data=('Poying', 23)
table_name='cats'
quer= f"INSERT INTO {table_name} {co} VALUES (%s, %s)"
quer

'INSERT INTO cats (name, age) VALUES (%s, %s)'

In [64]:
cursor.executemany(quer, data)

InterfaceError: Failed executing the operation; Could not process parameters: str(Poying), it must be of type list, tuple or dict

In [61]:
conn.commit()

In [19]:
conn.close()

In [22]:
cursor.close()

True